The purpose of this notebook is to demonstrate analysis conducted on cleaned and normalized data. We will seek to supplement the data found in existing report. Additionally, we will take a brief look at additional analytics that may strengthen analysis/provide impact metrics.

In [1]:
# EDA DB imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [2]:
# .Env variables and DB Connection
load_dotenv()

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')
db = os.getenv('DB_NAME')

conn_str = f'mssql+pymssql://{user}:{password}@{host}:{port}/{db}'
engine = create_engine(conn_str)

- The above would be 

In [3]:
# Create dataset for visual
query = """
SELECT 
    YEAR(timestamp) as year,
    MONTH(timestamp) as month,
    COUNT(*) as responses
FROM client_satisfaction
GROUP BY YEAR(timestamp), MONTH(timestamp)
ORDER BY year, month
"""

df = pd.read_sql(query, engine)

df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))


In [4]:
# Create interactive visualization with plotly
import plotly.express as px
import plotly.graph_objects as go

# Basic interactive bar chart
fig = px.bar(df, 
             x='date', 
             y='responses',
             title='Interactive Survey Response Timeline',
             labels={'date': 'Month', 'responses': 'Number of Responses'},
             hover_data={'year': True, 'month': True})

# Add mean line
fig.add_hline(y=df['responses'].mean(), 
              line_dash="dash", 
              line_color="red",
              annotation_text=f"Mean: {df['responses'].mean():.1f}")

# Customize layout
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Number of Responses',
    hovermode='x unified',
    height=600,
    showlegend=True
)

# Add text on bars
fig.update_traces(texttemplate='%{y}', textposition='outside')

fig.show()